# Event Streaming

--- IGNORE -- 

In this notebook, we'll see how to combine agents with callbacks to achieve **token by token streaming** from the underlying tools! We'll only
stream the tokens from the underlying tools and **nothing else**! Feel free to adapt this to your application needs.

Our agent will use the OpenAI tools API for tool invocation, and we'll provide the agent with two tools:

1. `where_cat_is_hiding`: A tool that uses an LLM to tell us where the cat is hiding
2. `tell_me_a_joke_about`: A tool that can use an LLM to tell a joke about the given topic

In [1]:
from typing import TYPE_CHECKING, Any, Dict, List, Optional, Sequence, TypeVar, Union
from uuid import UUID

from langchain import agents, hub
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.callbacks import Callbacks
from langchain_core.callbacks.base import AsyncCallbackHandler
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage
from langchain_core.outputs import ChatGenerationChunk, GenerationChunk, LLMResult
from langchain_openai import ChatOpenAI

In [2]:
from langchain_core.callbacks.manager import CallbackManager

## Create the model

**Attention** For older versions of langchain, we must set `streaming=True`

In [3]:
model = ChatOpenAI(temperature=0, streaming=True)

## Tools

We define two tools that rely on a chat model to generate output!

Please note a few different things:

1. We invoke the model using .stream() to force the output to stream (unfortunately for older langchain versions you should still set `streaming=True` on the model)
2. We attach tags to the model so that we can filter on said tags in our callback handler
3. The tools accept callbacks and propagate them to the model as a runtime argument

In [59]:
@tool
async def where_cat_is_hiding(callbacks: Callbacks) -> str:  # <--- Accept callbacks
    """Where is the cat hiding right now?"""
    chunks = [
        chunk
        async for chunk in model.astream(
            "Give one up to three word answer about where the cat might be hiding in the house right now.",
            {
                "tags": ["tool_llm"],
                "callbacks": callbacks,
            },  # <--- Propagate callbacks and assign a tag to this model
        )
    ]
    return "".join(chunk.content for chunk in chunks)


@tool
async def tell_me_a_joke_about(
    topic: str, callbacks: Callbacks
) -> str:  # <--- Accept callbacks
    """Tell a joke about a given topic."""
    template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are Cat Agent 007. You are funny and know many jokes."),
            ("human", "Tell me a long joke about {topic}"),
        ]
    )
    chain = template | model.with_config({"tags": ["tool_llm"]})
    chunks = [
        chunk
        async for chunk in chain.astream({"topic": topic}, {"callbacks": callbacks})
    ]
    return "".join(chunk.content for chunk in chunks)

## Initialize the Agent

In [60]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
print(prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [61]:
tools = [tell_me_a_joke_about, where_cat_is_hiding]
agent = agents.create_openai_tools_agent(
    model.with_config({"tags": ["agent"]}), tools, prompt
)
executor = agents.AgentExecutor(agent=agent, tools=tools)

In [62]:
async for event in executor.astream_events(
    {"input": "where is the cat hiding? Tell me a joke about that location?"},
    include_tags=["tool_llm"],
    include_types=["tool"],
):
    kind = event["event"]
    if kind == "on_llm_stream":
        print(event["data"]["chunk"].content, end="")
    if kind == "on_llm_end":
        print()
    if kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input', {})}"
        )  # Fix bug with empty events should be empty dict
    if kind == "on_tool_end":
        print(f"Ended tool: {event['name']}")
        print("--")

--
Starting tool: where_cat_is_hiding with inputs: {}
Under the bed.
Ended tool: where_cat_is_hiding
--
--
Starting tool: tell_me_a_joke_about with inputs: {'topic': 'under the bed'}
Sure, here's a long joke about what's hiding under the bed:

Once upon a time, there was a mischievous little boy named Timmy. Timmy had always been afraid of what might be lurking under his bed at night. Every evening, he would tiptoe into his room, turn off the lights, and then make a daring leap onto his bed, ensuring that nothing could grab his ankles.

One night, Timmy's parents decided to play a prank on him. They hid a remote-controlled toy monster under his bed, complete with glowing eyes and a growling sound effect. As Timmy settled into bed, his parents quietly snuck into his room, ready to give him the scare of a lifetime.

Just as Timmy was about to drift off to sleep, he heard a faint growl coming from under his bed. His eyes widened with fear, and his heart started racing. He mustered up the 